In [5]:
import os
import shutil
from monai.utils import first, set_determinism, progress_bar
import SimpleITK as sitk  # noqa: N813
import numpy as np
import itk
import tempfile
import monai
from monai.data import PILReader
from monai.transforms import (LoadImage, LoadImaged,ScaleIntensityRangePercentiles, Resized, Compose, SaveImage, 
ScaleIntensity,RandScaleIntensity, SpatialPadd)
from monai.config import print_config
from monai.data import CacheDataset, NiftiSaver
import glob
import torch
import nibabel as nib
from monai.transforms import (
    Activations,
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    NormalizeIntensityd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    RandAffined,
    SpatialPadd,
    Rand3DElasticd,
    RandFlipd,
    ResizeWithPadOrCropd
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss, DiceCELoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch, PatchDataset
from monai.config import print_config
from monai.apps import download_and_extract
from monai.inferers import sliding_window_inference
import matplotlib.pyplot as plt

In [6]:
cd

/home/athurai3


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    dropout = 0.4,
    norm=Norm.BATCH,
).to(device)

In [12]:
test_ct = ['projects/ctb-akhanf/athurai3/unet_contact_seg/sample_data/sub-P003/sub-P003_desc-rigid_space-T1w_znorm_ct.nii.gz']

test_data = [{"image": image} for image in test_ct]

test_transforms = Compose( #loading full image
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"])])

test_ds = Dataset(data = test_data, transform = test_transforms)

test_loader = DataLoader(test_ds, batch_size = 1, num_workers = 1)


post_transforms = post_transforms = Compose(
    [
        Activations(sigmoid = True),
        AsDiscrete(threshold = 0.5),
        #SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir=monai_test_dir, resample=False),
    ]
)
post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
date = 'Feb16'
pred_imgs = []
model.load_state_dict(torch.load(f'/home/athurai3/scratch/monai_outputs/UNET/{date}/checkpoint.pt', map_location=torch.device('cpu')))
model.eval()
with torch.no_grad():
    for test_img in test_loader:
        test_inputs = test_img['image'].to('cpu')
        roi_size = (32, 32, 32)
        sw_batch_size = 5
        pred_img = sliding_window_inference(inputs = test_inputs, roi_size=roi_size, sw_batch_size = sw_batch_size, predictor = model.to('cpu'),
                                            overlap = 0.15, mode = 'constant', sw_device = 'cpu', device = 'cpu', progress=True)
        pred_imgs.append(pred_img)

100%|██████████| 650/650 [05:33<00:00,  1.95it/s]


In [13]:
pred_imgs = post_transforms(pred_imgs)

In [14]:
pred_imgs.shape

AttributeError: 'list' object has no attribute 'shape'

In [8]:
prediction = pred_imgs[0][0][0]
file_name_pred = 'scratch/sub-P003_desc-rigid_space-T1w_ct_pred.nii.gz'
orig_test = nib.load('projects/ctb-akhanf/athurai3/unet_contact_seg/sample_data/sub-P003/sub-P003_desc-rigid_space-T1w_znorm_ct.nii.gz')
file = nib.Nifti1Image(prediction.detach().numpy(), affine = orig_test.affine, header = orig_test.header)
nib.save(file, file_name_pred)

In [4]:

test_dir = '/scratch/athurai3/preproc_outputs/test_data'
subjects = [identifier for identifier in os.listdir(test_dir) if "sub-" in identifier]
subjects = sorted(subjects)
subjects

['sub-P100',
 'sub-P104',
 'sub-P105',
 'sub-P106',
 'sub-P107',
 'sub-P109',
 'sub-P110',
 'sub-P111',
 'sub-P112',
 'sub-P113',
 'sub-P114']